In [278]:
import numpy as np
import sqlite3
import msgpack
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt

'''
Read data from sqlite database
'''
conn = sqlite3.connect('/home/a/LiCoRICE/decoder_training/participant_data/sabar_data.db')
cursor = conn.cursor()
spike_data = []
stick_internal = []
stick_decode = []
bin_width = []
task_params = []
task_state = []

for row in cursor.execute('SELECT * FROM signals_table'):
    stick_internal += [msgpack.unpackb(row[0], use_list=False)]
    spike_data += [msgpack.unpackb(row[1], use_list=False)]
    stick_decode += [msgpack.unpackb(row[2],use_list=False)]
    bin_width += [msgpack.unpackb(row[3],use_list=False)]
    task_params += [msgpack.unpackb(row[4],use_list=False)]
    task_state += [msgpack.unpackb(row[5],use_list=False)]
    
conn.close()

z = np.zeros((len(spike_data), len(spike_data[0])))
v_actual = np.zeros((len(stick_internal), len(stick_internal[0])))
v_decoded = np.zeros((len(stick_decode), len(stick_decode[0])))
bin_width = np.array(bin_width).flatten()
state = np.zeros((len(task_state),len(task_state[0])))

for i in range(len(spike_data)):
    z[i] = np.array(spike_data[i])
    v_actual[i] = np.array(stick_internal[i])
    v_decoded[i] = np.array(stick_decode[i])
    state[i] = np.array(task_state[i])


In [279]:
d = {'v_actual_x' : v_actual[:,0], 
     'v_actual_y' : v_actual[:,1],
     'v_decoded_x' : v_decoded[:,0],
     'v_decoded_y' : v_decoded[:,1],
    'bin_width' : bin_width[:],
    'target_x' : state[:,0],
    'target_y' : state[:,1],
    'cursor_x' : state[:,2],
    'cursor_y' : state[:,3],
    'in_trial' : state[:,4],
    'success' : state[:,5]}
df = pd.DataFrame(data=d)
idx = pd.TimedeltaIndex(df.index, unit='ms')
df = df.set_index(idx)

In [280]:
df['bin_width'] = df['bin_width'].astype('category')
df['in_trial'] = df['in_trial'].astype('category')
df['success'] = df['success'].astype('category')

In [289]:
len(df[df['success']==2])

108

In [282]:
# df_10 = df[df['bin_width']==10]
# df_25 = df[df['bin_width']==25]
# df_50 = df[df['bin_width']==50]
# df_100 = df[df['bin_width']==100]
# df_200 = df[df['bin_width']==200]
# df_300 = df[df['bin_width']==300]

# dfs = [df_10,df_25,df_50,df_100,df_200,df_300]

bin_widths = [17,35,75,150,300]

In [283]:
from ipywidgets import interact, fixed
import ipywidgets as widgets
%matplotlib inline

In [284]:
def plot_reaches(bin_width):
    bw = int(bin_width)
    
    d = df[df['bin_width'] == bw]
    x = np.arange(len(d['v_actual_x']))
    
    plt.figure(1)
    plt.subplot(211)
    plt.plot(x,d['v_actual_x'][:],'r',x,d['v_decoded_x'][:],'b')
    plt.subplot(212)
    plt.plot(x,d['v_actual_y'][:],'r',x,d['v_decoded_y'][:],'b')
    
    plt.show()

In [285]:
interact(plot_reaches, bin_width = bin_widths)

interactive(children=(Dropdown(description=u'bin_width', options=(17, 35, 75, 150, 300), value=17), Output()), _dom_classes=('widget-interact',))

<function __main__.plot_reaches>

In [246]:
# for n,df in enumerate(dfs):

#     i = 0
#     while df['in_trial'][i]:
#         i += 1

#     j = -1
#     while df['in_trial'][j]:
#         j -= 1

#     dfs[n] = df[i:j]

# df_10,df_25,df_50,df_100,df_200,df_300 = dfs
    